# Lambda Expressions in Java
## What Are Lambda Expressions?

Lambda expressions are a concise way to represent anonymous functions in Java. They allow you to treat functionality as a method argument, essentially letting you pass code as data. Lambda expressions are particularly useful for replacing anonymous classes when implementing interfaces that contain only one method (functional interfaces).

**Key Benefits:**
- More concise than anonymous classes
- Treat functionality as method arguments
- Cleaner, more readable code
- Enable functional programming concepts in Java

## Real-World Use Case: Social Networking Application

To understand lambda expressions, let's follow a practical example of a social networking application where an administrator needs to perform actions on members based on specific criteria.

### The Person Class
```java
public class Person {
    public enum Sex { MALE, FEMALE }
    
    String name;
    LocalDate birthday;
    Sex gender;
    String emailAddress;
    
    public int getAge() { /* implementation */ }
    public void printPerson() { /* implementation */ }
}
```

## Evolution from Basic Methods to Lambda Expressions

### Approach 1: Basic Method (Rigid)
```java
public static void printPersonsOlderThan(List<Person> roster, int age) {
    for (Person p : roster) {
        if (p.getAge() >= age) {
            p.printPerson();
        }
    }
}
```
**Problem:** Too specific, creates brittle code that breaks when requirements change.

### Approach 2: More Generic Method (Still Limited)
```java
public static void printPersonsWithinAgeRange(
    List<Person> roster, int low, int high) {
    for (Person p : roster) {
        if (low <= p.getAge() && p.getAge() < high) {
            p.printPerson();
        }
    }
}
```
**Problem:** Still requires new methods for each different search criteria.

### Approach 3: Using Local Class (Better Separation)
```java
interface CheckPerson {
    boolean test(Person p);
}

class CheckPersonEligibleForSelectiveService implements CheckPerson {
    public boolean test(Person p) {
        return p.gender == Person.Sex.MALE &&
            p.getAge() >= 18 && p.getAge() <= 25;
    }
}

public static void printPersons(List<Person> roster, CheckPerson tester) {
    for (Person p : roster) {
        if (tester.test(p)) {
            p.printPerson();
        }
    }
}
```
**Usage:**
```java
printPersons(roster, new CheckPersonEligibleForSelectiveService());
```

### Approach 4: Anonymous Class (Reduced Code)
```java
printPersons(
    roster,
    new CheckPerson() {
        public boolean test(Person p) {
            return p.getGender() == Person.Sex.MALE
                && p.getAge() >= 18
                && p.getAge() <= 25;
        }
    }
);
```
**Problem:** Still bulky syntax for a simple operation.

### Approach 5: Lambda Expression (Concise Solution)
```java
printPersons(
    roster,
    (Person p) -> p.getGender() == Person.Sex.MALE
        && p.getAge() >= 18
        && p.getAge() <= 25
);
```
**This is much cleaner!** The lambda expression `(Person p) -> ...` replaces the entire anonymous class.

## Standard Functional Interfaces

### Approach 6: Using Predicate<T>
Instead of creating custom interfaces, Java provides standard functional interfaces:

```java
import java.util.function.Predicate;

public static void printPersonsWithPredicate(
    List<Person> roster, Predicate<Person> tester) {
    for (Person p : roster) {
        if (tester.test(p)) {
            p.printPerson();
        }
    }
}

// Usage with lambda
printPersonsWithPredicate(
    roster,
    p -> p.getGender() == Person.Sex.MALE
        && p.getAge() >= 18
        && p.getAge() <= 25
);
```

### Approach 7: Multiple Lambda Expressions
Using `Consumer<T>` for actions:

```java
import java.util.function.Consumer;

public static void processPersons(
    List<Person> roster,
    Predicate<Person> tester,
    Consumer<Person> block) {
    for (Person p : roster) {
        if (tester.test(p)) {
            block.accept(p);
        }
    }
}

// Usage
processPersons(
    roster,
    p -> p.getGender() == Person.Sex.MALE && p.getAge() >= 18 && p.getAge() <= 25,
    p -> p.printPerson()  // Lambda for the action
);
```

### Approach 8: Using Function<T,R> for Data Transformation
```java
import java.util.function.Function;

public static void processPersonsWithFunction(
    List<Person> roster,
    Predicate<Person> tester,
    Function<Person, String> mapper,
    Consumer<String> block) {
    for (Person p : roster) {
        if (tester.test(p)) {
            String data = mapper.apply(p);
            block.accept(data);
        }
    }
}

// Extract email addresses
processPersonsWithFunction(
    roster,
    p -> p.getGender() == Person.Sex.MALE && p.getAge() >= 18 && p.getAge() <= 25,
    p -> p.getEmailAddress(),  // Transform Person to String
    email -> System.out.println(email)  // Print the email
);
```

### Approach 9: Generic Version
```java
public static <X, Y> void processElements(
    Iterable<X> source,
    Predicate<X> tester,
    Function<X, Y> mapper,
    Consumer<Y> block) {
    for (X p : source) {
        if (tester.test(p)) {
            Y data = mapper.apply(p);
            block.accept(data);
        }
    }
}
```

## Stream API with Lambda Expressions

The most modern approach uses Java's Stream API:

```java
roster.stream()
    .filter(p -> p.getGender() == Person.Sex.MALE 
             && p.getAge() >= 18 
             && p.getAge() <= 25)
    .map(p -> p.getEmailAddress())
    .forEach(email -> System.out.println(email));
```

**Mapping of Operations:**
| Traditional Method | Stream Operation |
|-------------------|------------------|
| Get source | `stream()` |
| Filter with Predicate | `filter()` |
| Transform with Function | `map()` |
| Act with Consumer | `forEach()` |

## Lambda Syntax

### Basic Structure
```
(parameters) -> expression
```
or
```
(parameters) -> { statements; }
```

### Examples
```java
// No parameters
() -> System.out.println("Hello")

// One parameter (parentheses optional)
p -> p.getAge() > 18
(p) -> p.getAge() > 18

// Multiple parameters
(a, b) -> a + b

// With explicit types
(Person p) -> p.getAge() > 18

// Block body
p -> {
    System.out.println("Processing: " + p.getName());
    return p.getAge() > 18;
}
```

### Calculator Example
```java
public class Calculator {
    interface IntegerMath {
        int operation(int a, int b);   
    }
  
    public int operateBinary(int a, int b, IntegerMath op) {
        return op.operation(a, b);
    }
 
    public static void main(String... args) {
        Calculator myApp = new Calculator();
        
        // Lambda expressions for different operations
        IntegerMath addition = (a, b) -> a + b;
        IntegerMath subtraction = (a, b) -> a - b;
        
        System.out.println("40 + 2 = " + myApp.operateBinary(40, 2, addition));
        System.out.println("20 - 10 = " + myApp.operateBinary(20, 10, subtraction));
    }
}
```

## GUI Applications with Lambda Expressions

### Before (Anonymous Class)
```java
btn.setOnAction(new EventHandler<ActionEvent>() {
    @Override
    public void handle(ActionEvent event) {
        System.out.println("Hello World!");
    }
});
```

### After (Lambda Expression)
```java
btn.setOnAction(event -> System.out.println("Hello World!"));
```

## Variable Scope and Access

Lambda expressions have lexical scoping, meaning they don't create a new scope level:

```java
public class LambdaScopeTest {
    public int x = 0;

    class FirstLevel {
        public int x = 1;
        
        void methodInFirstLevel(int x) {
            int z = 2;
             
            Consumer<Integer> myConsumer = (y) -> {
                System.out.println("x = " + x);                    // method parameter
                System.out.println("y = " + y);                    // lambda parameter
                System.out.println("z = " + z);                    // local variable
                System.out.println("this.x = " + this.x);          // FirstLevel.x
                System.out.println("LambdaScopeTest.this.x = " +   // outer class
                    LambdaScopeTest.this.x);
            };
 
            myConsumer.accept(x);
        }
    }
}
```

### Important Rules:
1. **No shadowing**: Lambda parameters cannot redeclare local variables from enclosing scope
2. **Effectively final**: Can only access local variables that are final or effectively final
3. **Lexical scoping**: No new scope level created

## Target Typing

The type of a lambda expression is determined by the context (target type):

```java
// Same lambda, different types based on context
Predicate<Person> predicate = p -> p.getAge() > 18;      // Predicate<Person>
CheckPerson checker = p -> p.getAge() > 18;              // CheckPerson
```

### Method Overloading Example
```java
void invoke(Runnable r) { r.run(); }
<T> T invoke(Callable<T> c) { return c.call(); }

// This calls invoke(Callable<T>) because it returns a value
String s = invoke(() -> "done");
```

## Key Functional Interfaces

| Interface | Method | Purpose | Example |
|-----------|--------|---------|---------|
| `Predicate<T>` | `boolean test(T t)` | Testing/filtering | `p -> p.getAge() > 18` |
| `Consumer<T>` | `void accept(T t)` | Consuming/acting on data | `p -> System.out.println(p)` |
| `Function<T,R>` | `R apply(T t)` | Transforming data | `p -> p.getEmailAddress()` |
| `Supplier<T>` | `T get()` | Supplying data | `() -> new Person()` |

## Best Practices

1. **Keep lambda expressions short and simple**
2. **Use method references when possible**: `System.out::println` instead of `x -> System.out.println(x)`
3. **Prefer standard functional interfaces over custom ones**
4. **Use descriptive parameter names**
5. **Consider readability over brevity**

## Summary

Lambda expressions transform Java from purely object-oriented to supporting functional programming concepts. They provide:

- **Concise syntax** for anonymous functions
- **Better readability** for simple operations  
- **Functional programming** capabilities
- **Seamless integration** with Stream API
- **Type inference** and target typing
- **Lexical scoping** without shadowing issues

The evolution from basic methods to lambda expressions with streams represents a fundamental shift toward more expressive, maintainable code in Java.